## Описание проекта

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## 1. Подготовка

In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import re
import sys

import spacy

import nltk
from nltk.corpus import stopwords

import gensim.models
from gensim.test.utils import datapath
from gensim import utils

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

from catboost import Pool
from catboost import CatBoostClassifier

In [3]:
# если необходимо установить spacy
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m spacy download en_core_web_sm

Прочитаем данные из файла и выведем первые 5 строк.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#comments = pd.read_csv('/datasets/toxic_comments.csv')
comments = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml_for_texts/toxic_comments.csv')
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Посмотрим общую информацию о таблице.

In [6]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропущенных строк нет, колонки названы корректно, типы данных определены верно. Обрабатываем дальше.

Напишем функцию для очистки текста от лишних символов. Используем для это регулярные выражения. Оставим в тексте только латинские буквы(в обоих регистрах) и пробелы. Остальное все заменим на пробелы.

In [7]:
def clear_text(text):
    cl_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    cl_text = ' '.join(cl_text.split())
    return cl_text

Сохраним значения в отдельной колонке.

In [8]:
comments['clear_text'] = comments['text'].apply(clear_text)

Напишем функцию для лемматизации текстов. Для этой цели будем использовать библиотеку spaCy.

In [9]:
def lemmatize_text(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    lemm_text = ' '.join([token.lemma_ for token in doc])
    return lemm_text

Проверим, как быстро у нас лемматизируется выборка из 100 случайных строк.

In [10]:
check_speed = comments.sample(100).reset_index(drop=True)

In [11]:
%%time
check_speed['lemm_text'] = check_speed['clear_text'].apply(lemmatize_text)

CPU times: user 51.4 s, sys: 1.04 s, total: 52.4 s
Wall time: 52.4 s


Видно, что лемматизация для 100 строк длится примерно минуту. У нас строк 160000 - и это займет не один час. Попробуем обучить модели на нелемматизированных строках. Однако посмотрим, работу данной функции и выполняет ли она лемматизацию верно.

In [12]:
print('clear_text:', check_speed['clear_text'][0])
print('-' * 40)
print('lemm_text:', check_speed['lemm_text'][0])

clear_text: About the Manhattan Bridge See Acps I read the history site on the W train page It says When all four tracks on the Manhattan Bridge were restored on February the W was changed to its final service pattern as a weekday local between Whitehall Street South Ferry and Queens This means the W was local and the N was express That s when the Manhattan Bridge s north tracks reopen for the B and D trains Read more about the service history of the B D N Q and W train pages so you can get a better understand you meanie D
----------------------------------------
lemm_text: about the Manhattan Bridge see Acps -PRON- read the history site on the w train page -PRON- say when all four track on the Manhattan Bridge be restore on February the W be change to -PRON- final service pattern as a weekday local between Whitehall Street South Ferry and Queens this mean the w be local and the n be express that s when the Manhattan Bridge s north track reopen for the b and d train read more about the

Видно, что операция выполняется. Стоит заметить, что вместо местоимений здесь подставляется -PRON-.

Дальше будем работать только с очищенными данными.

Разделим наши данные на тренировочную и тестовую выборки.

In [13]:
train, test = train_test_split(comments, test_size=0.2, random_state=12345)

Также получим целевые признаки для обучающей и тестовой выборок.

In [14]:
target_train = train['toxic']
target_test = test['toxic']

### TF-IDF

Создадим корпусы текстов, путем преобразования колонки clear_text в список текстов для обеих выборок.

In [15]:
corpus_train = list(train['clear_text'])
corpus_test = list(test['clear_text'])

Загрузим список стоп-слов.

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Получим английские стоп-слова.

In [17]:
stop_words = set(stopwords.words('english'))

Вычислим TF-IDF для корпуса текстов.

In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words, lowercase=True, min_df=0.001)
features_train_tf_idf = count_tf_idf.fit_transform(corpus_train)

На обученном TfidfVectorizer, получим признаки для тестовой выборки.

In [19]:
features_test_tf_idf = count_tf_idf.transform(corpus_test)

Посмотрим тип получившихся признаков.

In [20]:
type(features_train_tf_idf)

scipy.sparse.csr.csr_matrix

Подготовим признаки в табличном виде.

In [21]:
features_train_tf_idf_vec = pd.DataFrame(features_train_tf_idf.toarray())

In [22]:
features_test_tf_idf_vec = pd.DataFrame(features_test_tf_idf.toarray())

На данном шаге мы получили признаки, основанные на величинах TF-IDF для каждого слова.

### Word2Vec

Теперь переведем наши текста в вектора с помощью метода Word2Vec. Для этих целей будем использовать библиотеку `gensim`.

Чтобы она работала правильно, необходимо особым образом представить наши текста.<br> 
Из документации: *Note the sentences iterable must be restartable (not just a generator), to allow the algorithm to stream over your dataset multiple times.*

In [23]:
class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        
        for line in train['clear_text'].values:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

Укажем, через константу, сколько признаков через Word2Vec мы хотим получить.

In [24]:
NUM_WV_FEATURES = 100

Создадим нашу модель, которая будет береводить слова в вектора.

In [25]:
sentences = MyCorpus()
w2v = gensim.models.Word2Vec(sentences=sentences)

Напишем функцию, которая для переданной строки `row` будет формировать соответствующи вектор с количесвтом признаков равным `num_features`. Сначала мы для каждого слова отдельно найдем соответсвующий вектор, а потом сложим все вектора для слов и получим вектор для предложения.

In [26]:
def get_vectors(row):
    vecs = [np.zeros(NUM_WV_FEATURES)]
    for w in utils.simple_preprocess(row['clear_text']):
        try:
            v = w2v[w]
        except:
            v = np.zeros(NUM_WV_FEATURES)
        vecs.append(v)

    return np.sum(np.array(vecs), axis=0)

Применим функцию для тренировочной выборки.

In [27]:
train.head()

,text,toxic,clear_text
45800,I have offered as well...but he must do my map...,0,I have offered as well but he must do my map f...
94209,YOUR biased! ORTHODOX VIEWS have no more merit...,0,YOUR biased ORTHODOX VIEWS have no more merit ...
135210,Thank you very much for your quick respond and...,0,Thank you very much for your quick respond and...
89158,"I moved this page. When I did, I found anothe...",0,I moved this page When I did I found another l...
61233,Gray Powell article nominated for deletion \nN...,0,Gray Powell article nominated for deletion Nom...


In [28]:
train['wv'] = train.apply(get_vectors, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Посмотрим, как отработала данная функция.

In [29]:
train.head()

,text,toxic,clear_text,wv
45800,I have offered as well...but he must do my map...,0,I have offered as well but he must do my map f...,"[1.8095783032476902, -12.22042335383594, 6.460..."
94209,YOUR biased! ORTHODOX VIEWS have no more merit...,0,YOUR biased ORTHODOX VIEWS have no more merit ...,"[8.39693272486329, -57.38118848018348, -16.702..."
135210,Thank you very much for your quick respond and...,0,Thank you very much for your quick respond and...,"[-3.331927366554737, -53.02797158062458, 3.450..."
89158,"I moved this page. When I did, I found anothe...",0,I moved this page When I did I found another l...,"[11.90593901090324, 6.0668970271945, 5.4115197..."
61233,Gray Powell article nominated for deletion \nN...,0,Gray Powell article nominated for deletion Nom...,"[11.662838727235794, 1.9756474867463112, 14.75..."


Подготовим признаки в табличном виде.

In [30]:
features_train_wv_vec = pd.DataFrame(train['wv'].to_list())

Выполнил теже самые операции для тестовой выборки.

In [31]:
test['wv'] = test.apply(get_vectors, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
features_test_wv_vec = pd.DataFrame(test['wv'].to_list())

На данном шаге мы получили признаки, основанные на векторах (эмбеддингах).

### Вывод

На данном этапе мы подготовили данные для обучения. Загрузили данные из файла. Посмотрели, что нет пропущенных строк, типы данных определены правильно и названия колонок корректные. Далее мы почистили данные от лишних символов. Проводить лемматизацию мы проводить не стали, потому что это может занять много времени, попробуем обучиться только на очищенных данных. Далее мы разделили первоначальные данные на тренировочную и тестовую выборки. Затем подоготовили признаки для обучения. Это мы сделали двумя способами: путем подсчета значений TF-IDF для каждого слова и путем преобразования строк в вектора спомощью метода Word2Vec.

## 2. Обучение

Рассмотрим несколько моделей для классификации комментариев на позитивные и негативные. Сначала обучим их на тренировочной выборке, а затем оценим качество на тестовой с помощью метрики F1. У нас есть несколько наборов признаков полученных из данных, каждую модель попробуем обучить на этих признаках.

### 2.1 Логистическая регрессия

Рассмотрим модель логитической регрессии.

#### TF-IDF

Обучим модель на признаках TF-IDF.

In [33]:
model_lr_tf_idf = LogisticRegression(max_iter=1000, random_state=12345)

In [34]:
model_lr_tf_idf.fit(features_train_tf_idf_vec, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12345, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
predictions_lr_tf_idf = model_lr_tf_idf.predict(features_test_tf_idf_vec)

In [36]:
f1_lr_tf_idf = f1_score(target_test, predictions_lr_tf_idf)
f1_lr_tf_idf

0.727103845439346

#### Word2Vec

Обучим на векторах.

In [37]:
model_lr_wv = LogisticRegression(max_iter=1000, random_state=12345)

In [38]:
model_lr_wv.fit(features_train_wv_vec, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12345, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
predictions_lr_wv = model_lr_wv.predict(features_test_wv_vec)

In [40]:
f1_lr_wv = f1_score(target_test, predictions_lr_wv)
f1_lr_wv

0.5536560247167868

### 2.2 CatBoost

Рассмотри модель из библиотека CatBoost

#### Text features

Из коробки, CatBoost умеет работать с текстовыми признаками. Помимо подготовленных признаков, проверим, как она будет работать через этот подход.

In [41]:
model_cbc_default_tf = CatBoostClassifier(eval_metric='F1', iterations=2000, verbose=100, random_state=12345, task_type='GPU')

In [42]:
features_col = ['clear_text']
target_col = ['toxic']
text_features = ['clear_text']

In [43]:
model_cbc_default_tf.fit(train[features_col], 
                         train[target_col], 
                         text_features=text_features)

Learning rate set to 0.013875
0:	learn: 0.6980083	total: 20.6ms	remaining: 41.2s
100:	learn: 0.6904691	total: 1.81s	remaining: 34.1s
200:	learn: 0.7023369	total: 3.56s	remaining: 31.9s
300:	learn: 0.7116819	total: 5.28s	remaining: 29.8s
400:	learn: 0.7184638	total: 6.98s	remaining: 27.8s
500:	learn: 0.7237718	total: 8.67s	remaining: 25.9s
600:	learn: 0.7297013	total: 10.4s	remaining: 24.1s
700:	learn: 0.7341409	total: 12.1s	remaining: 22.4s
800:	learn: 0.7374741	total: 13.8s	remaining: 20.6s
900:	learn: 0.7390290	total: 15.5s	remaining: 18.9s
1000:	learn: 0.7429250	total: 17.2s	remaining: 17.1s
1100:	learn: 0.7448264	total: 18.8s	remaining: 15.4s
1200:	learn: 0.7465873	total: 20.6s	remaining: 13.7s
1300:	learn: 0.7488079	total: 22.3s	remaining: 12s
1400:	learn: 0.7505647	total: 24s	remaining: 10.3s
1500:	learn: 0.7516527	total: 25.7s	remaining: 8.54s
1600:	learn: 0.7531813	total: 27.4s	remaining: 6.83s
1700:	learn: 0.7547428	total: 29.1s	remaining: 5.11s
1800:	learn: 0.7556449	total: 3

In [44]:
predictions_cbc_default_tf = model_cbc_default_tf.predict(test[features_col])

In [45]:
f1_cbc_default_tf = f1_score(target_test, predictions_cbc_default_tf)
f1_cbc_default_tf

0.7550669872895912

#### TF-IDF

Рассмотрим признаки на основе величин TF-IDF.

In [46]:
model_cbc_tf_idf = CatBoostClassifier(eval_metric='F1', iterations=2000, verbose=100, random_state=12345, task_type='GPU')

In [47]:
model_cbc_tf_idf.fit(features_train_tf_idf_vec, target_train)

Learning rate set to 0.013875
0:	learn: 0.5213286	total: 66.9ms	remaining: 2m 13s
100:	learn: 0.5080641	total: 5.54s	remaining: 1m 44s
200:	learn: 0.5397774	total: 10.7s	remaining: 1m 35s
300:	learn: 0.5678063	total: 15.9s	remaining: 1m 29s
400:	learn: 0.5918920	total: 21s	remaining: 1m 23s
500:	learn: 0.6121104	total: 26.1s	remaining: 1m 18s
600:	learn: 0.6354279	total: 31.2s	remaining: 1m 12s
700:	learn: 0.6495571	total: 36.2s	remaining: 1m 7s
800:	learn: 0.6638664	total: 41.3s	remaining: 1m 1s
900:	learn: 0.6689545	total: 46.3s	remaining: 56.4s
1000:	learn: 0.6751989	total: 51.3s	remaining: 51.2s
1100:	learn: 0.6851408	total: 56.4s	remaining: 46s
1200:	learn: 0.6925359	total: 1m 1s	remaining: 40.8s
1300:	learn: 0.6962755	total: 1m 6s	remaining: 35.6s
1400:	learn: 0.7018643	total: 1m 11s	remaining: 30.5s
1500:	learn: 0.7060629	total: 1m 16s	remaining: 25.4s
1600:	learn: 0.7089116	total: 1m 21s	remaining: 20.3s
1700:	learn: 0.7114312	total: 1m 26s	remaining: 15.2s
1800:	learn: 0.71541

In [48]:
predictions_cbc_tf_idf = model_cbc_tf_idf.predict(features_test_tf_idf_vec)

In [49]:
f1_cbc_tf_idf = f1_score(target_test, predictions_cbc_tf_idf)
f1_cbc_tf_idf

0.7064865893094922

#### Word2Vec

Для данной модели на этих признаках получается небольшое значение метрики F1. Из-за того, что появились проблемы с оперативной памятью, было решено не рассматривать данный случай, а направить ресурсы на проверку работы модели BERT.

Обучим на векторах.

In [50]:
#model_cbc_wv = CatBoostClassifier(eval_metric='F1', iterations=2000, verbose=100, random_state=12345, task_type='GPU')

In [51]:
#model_cbc_wv.fit(features_train_wv_vec, target_train)

In [52]:
#predictions_cbc_wv = model_cbc_wv.predict(features_test_wv_vec)

In [53]:
#f1_cbc_wv = f1_score(target_test, predictions_cbc_wv)
#f1_cbc_wv

### 2.3 BERT

Кроме всего прочего попробуем получить эмбеддинги с помощью модели BERT. Подключим необходимые модули здесь, чтобы удобнее было работать с моделью.

In [54]:
!pip install transformers

In [55]:
import torch
import transformers
from tqdm import notebook

Скопируем нашу таблицу для работы только с BERT

In [56]:
comments_bert = comments.copy()

В дальнейшем мы будем работать с тензорами, а также с предобученной моделью. Для нее уже устанвлен размер эмбеддингов 512. Поэтому напишем функцию, которая обрежет наши предложения до 510 слов.

In [57]:
def cut_text(row):
    row_list = row.split()
    row_list = row_list[:510]
    row_text = ' '.join(row_list)
    return row_text

Сохраним обрезанные признаки в отдельную колонку.

In [58]:
comments_bert['cut_clear_text'] = comments_bert['clear_text'].apply(cut_text)

Возьмем лишь небольшую часть данных, чтобы в дальнейшем хватило памяти и все выполнялось в разумное время.

In [59]:
short_comments_bert = comments_bert.sample(7000).reset_index(drop=True)

Из открытого репозитория DeepPavlov была взята предобученная модель Bert, обученная на корпусе английских текстов, учитывающая символы как в нижнем, так и в верхнем регистрах - `BERT-Base, Cased`

In [60]:
tokenizer = transformers.BertTokenizer(vocab_file='/content/drive/MyDrive/Colab Notebooks/ml_for_texts/cased_L-12_H-768_A-12/vocab.txt')

tokenized = short_comments_bert['cut_clear_text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, max_length=512, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [61]:
config = transformers.BertConfig.from_json_file(
    '/content/drive/MyDrive/Colab Notebooks/ml_for_texts/cased_L-12_H-768_A-12/bert_config.json')
model = transformers.BertForPreTraining.from_pretrained(
    '/content/drive/MyDrive/Colab Notebooks/ml_for_texts/cased_L-12_H-768_A-12/bert_model.ckpt.index', config=config, from_tf=True)

Выберем устройство, на котором будут происходить вычисления.

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Выполним дополнительные операции для работы с GPU.

In [63]:
torch.cuda.empty_cache()

Получим эмбеддинги

In [64]:
batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
        
        with torch.no_grad():
            model = model.to(device)
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

Определим признаки, разделим на обучающую и тестовую выборки.

In [65]:
features_bert = np.concatenate(embeddings)
target_bert = short_comments_bert['toxic']

In [66]:
features_bert_train, features_bert_test, target_bert_train, target_bert_test = \
        train_test_split(features_bert, target_bert, test_size=0.2, random_state=12345)

Обучим логистическую регрессию и посмотрим на качество модели.

In [67]:
model_lr_bert = LogisticRegression()
model_lr_bert.fit(features_bert_train, target_bert_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
predictions_lr_bert = model_lr_bert.predict(features_bert_test)

In [69]:
f1_lr_bert = f1_score(target_bert_test, predictions_lr_bert)
f1_lr_bert

0.6609442060085838

### Вывод

На данном шаге мы обучали различные модели на различных признаках и оценивали их работу с помощью метрики F1. В качестве моделей мы использовали Логистическую регрессию и CatBoost. Также применили модель BERT, с помощью предобученной модели получили эмбеддинги, которые использовали в качестве признаков для логистической регрессии.<br>
f1_lr_tf_idf = 0.72710<br>
f1_lr_wv = 0.55365<br>
f1_cbc_default_tf = 0.75507<br>
f1_cbc_tf_idf = 0.70648<br>
f1_lr_bert = 0.66094<br>
Лучшей моделью в нашем случае стал CatBoost, который обучался на необработанных тектовых признаках - значение метрики 0.75507.<br>
Невысокие значения остальных моделей можно объяснить тем, что мы не проводили лемматизацию строк, а лишь очистили от небуквенных символов.<br>
Значение такой метрики F1 на модели BERT, можно обосновать тем, что во-первых, также не была проведена лемматизация; во вторых, мы обрезали тексты до 512 слов, чтобы предобученная модель работала корректно; в-третьих, что мы создавали эмбеддинги только для части данных, чтобы уложиться в память и во разумное время.

## 3. Общий вывод

В данной работе мы научились классифицировать комментарии на позитивные и негативные. Научились обрабатывать текст, проводить очистку и лемматизацию. Также освоили преобразование текстов в числовое представление с помощью вычисления значений TF-IDF, в векторное представление (эмбеддинги) с помощью сетода Word2Vec. Обучили на полученных признаках различные модели классификации. Также проверили работу модели BERT, которая с помощью предобучения на корпусе текстов, позволила преобразовать текст в эмбеддинги. По итогу, наилучшей стала модель CatBoost, которая использовала для обучения встроенную поддержку текстовых признаков. 